In [20]:
%pip install sqlite-vec


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import sqlite3
import sqlite_vec

db = sqlite3.connect("memory-test-vector.db")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

(vec_version,) = db.execute("select vec_version()").fetchone()
print(f"vec_version={vec_version}")

vec_version=v0.1.6


In [13]:
from sqlite_vec import serialize_float32

embedding = [0.1, 0.2, 0.3, 0.4]
result = db.execute("select vec_length(?)", [serialize_float32(embedding)])

print(result.fetchone()[0])  # 4

4


In [14]:
from typing import List
import struct


def serialize_f32(vector: List[float]) -> bytes:
    """serializes a list of floats into a compact "raw bytes" format"""
    return struct.pack("%sf" % len(vector), *vector)

# memory sqlite db
#db = sqlite3.connect(":memory:")
#db.enable_load_extension(True)
#sqlite_vec.load(db)
#db.enable_load_extension(False)

sqlite_version, vec_version = db.execute(
    "select sqlite_version(), vec_version()"
).fetchone()
print(f"sqlite_version={sqlite_version}, vec_version={vec_version}")

items = [
    (1, [0.1, 0.1, 0.1, 0.1]),
    (2, [0.2, 0.2, 0.2, 0.2]),
    (3, [0.3, 0.3, 0.3, 0.3]),
    (4, [0.4, 0.4, 0.4, 0.4]),
    (5, [0.5, 0.5, 0.5, 0.5]),
]
query = [0.3, 0.3, 0.3, 0.3]

db.execute("CREATE VIRTUAL TABLE vec_items USING vec0(embedding float[4])")

with db:
    for item in items:
        db.execute(
            "INSERT INTO vec_items(rowid, embedding) VALUES (?, ?)",
            [item[0], serialize_f32(item[1])],
        )

rows = db.execute(
    """
      SELECT
        rowid,
        distance
      FROM vec_items
      WHERE embedding MATCH ?
      ORDER BY distance
      LIMIT 3
    """,
    [serialize_f32(query)],
).fetchall()

print(rows)

sqlite_version=3.41.1, vec_version=v0.1.6


OperationalError: table vec_items already exists

In [4]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("./env/.env"))

import dashscope
from http import HTTPStatus
from pprint import pprint

with open("来自 Google 内部的另外一种声音：AI 没有护城河.md", 'r') as f:
    text = f.read()

resp = dashscope.TextEmbedding.call(
    model=dashscope.TextEmbedding.Models.text_embedding_v2,
    input="We are lucky to live in an age in which we are still making discoveries.",
    dimension=1536,
)
pprint("ok") if resp.status_code == HTTPStatus.OK else print(resp)
pprint(resp["output"]["embeddings"][0]["embedding"][:3])

'ok'
[0.022378576171554372, -0.027432455162420308, -0.00355793080956962]


In [1]:
%pip install tiktoken


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
from itertools import islice
import tiktoken
import numpy as np

def batched(iterable, n):
    """将数据分批处理成每批长度为 n 的元组。最后一批的长度可能较短。"""
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError("n must be at least one")
    it = iter(iterable)
    while batch := islice(it, n):
        yield batch


def chunked_tokens(text, encoding_name, chunk_length):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    chunk_items_len = len(tokens) // chunk_length + 1
    print("chunk items length: ", chunk_items_len)
    text_len = len(text) // chunk_items_len + 1
    chunks_iterator = [text[i : i + text_len] for i in range(0, len(text), text_len)]
    yield from chunks_iterator


def len_safe_get_embedding(
    text,
    model="text-embedding-v2",
    max_tokens=2048,
    encoding_name="cl100k_base",
):
    chunk_embeddings = []
    chunk_lens = []
    for chunk in chunked_tokens(
        text, encoding_name=encoding_name, chunk_length=max_tokens
    ):
        print("length: ", len(chunk))
        chunk_embeddings.append(get_embedding(chunk, model=model))
        chunk_lens.append(len(chunk))

    chunk_embeddings = np.average(chunk_embeddings, axis=0, weights=chunk_lens)
    return chunk_embeddings

def get_embedding(text, model):
    resp = dashscope.TextEmbedding.call(
        model=model,
        input=text,
        dimension=1024,
    )
    return resp["output"]["embeddings"][0]["embedding"]


print(len(text))
tokens = tiktoken.get_encoding("cl100k_base").encode(text)
print(len(tokens))
chunked_embeddings = len_safe_get_embedding(text)
print(len(chunked_embeddings))

18182
5457
chunk items length:  3
length:  6061
length:  6061
length:  6060
1536


In [25]:
print(len(text))
resp_v3 = dashscope.TextEmbedding.call(
    model=dashscope.TextEmbedding.Models.text_embedding_v3,
    input=text,
    dimension=1024,
)
print(resp_v3["output"]["embeddings"][0]['embedding'][:5])
embeddings = resp_v3["output"]["embeddings"]
print(len(embeddings))

# resp_batched = dashscope.BatchTextEmbedding.call(
#    model=dashscope.BatchTextEmbedding.Models.text_embedding_async_v2,
#    url="https://help-static-aliyun-doc.aliyuncs.com/file-manage-files/zh-CN/20241016/nigwvr/text_embedding_file.txt",
#    text_type="document"
# )
# pprint(resp_batched)

18182
[-0.07153698801994324, -0.04376567527651787, -0.030815014615654945, -0.006778706330806017, -0.06811530888080597]
1


In [26]:
print(embeddings[0]["embedding"][:5])


[-0.07153698801994324, -0.04376567527651787, -0.030815014615654945, -0.006778706330806017, -0.06811530888080597]


In [18]:
db.execute(
    """CREATE VIRTUAL TABLE if NOT EXISTS vec_document_items USING vec0(
    id INTEGER PRIMARY KEY,
    document_name TEXT,
    embedding float[1024]
)"""
)

id = 1
name = "./vectordb/来自 Google 内部的另外一种声音：AI 没有护城河.md"
vector = embeddings[0]["embedding"]

db.execute(
    "INSERT INTO vec_document_items(id, document_name, embedding) VALUES (?, ?, ?)",
    [id, name, serialize_f32(vector)],
)
db.commit()

In [45]:
db.execute(
    """CREATE VIRTUAL TABLE if NOT EXISTS vec_chunked_document_items USING vec0(
    id INTEGER PRIMARY KEY,
    document_name TEXT,
    embedding float[1536]
)"""
)
with db:
    for id, embedding in enumerate(chunked_embeddings):
        db.execute(
            "INSERT INTO vec_chunked_document_items(id, document_name, embedding) VALUES (?, ?, ?)",
            [id, name, serialize_f32(embedding)],
        )

In [46]:
# insert again
with db:
    for id, embedding in enumerate(chunked_embeddings):
        db.execute(
            "INSERT INTO vec_chunked_document_items(id, document_name, embedding) VALUES (?, ?, ?)",
            [id, name, serialize_f32(embedding)],
        )

OperationalError: UNIQUE constraint failed on vec_chunked_document_items primary key